In [ ]:
import pandas as pd
from finx_ib_reports.input_sources import TwsApiSource

from ib_insync import util, IB

In [ ]:
util.startLoop()

In [ ]:
ib = IB()
ib.connect('127.0.0.1', 8888, clientId=3)

In [ ]:
similar_product_mapping = {
    "MCL": "MCL",
    "YC": "ZC"
}

s = TwsApiSource(ib)
aids = [x for x in s.get_accounts() if 'U31' in x]

for aid in aids:
    positions = s.get_positions(aid)

positions = [x for x in positions if x.contract.symbol in ("CL", "MCL")]    

In [ ]:
from ib_insync.objects import PnLSingle

for x in positions:
    ib.reqPnLSingle(account=aid, conId=x.contract.conId, modelCode="")
    # ib.cancelPnLSingle(account=aid, modelCode="", conId=x.contract.conId)

In [ ]:
ib.pnlSingle()

In [ ]:
import re

def format_col_names(cols):
    result = []
    for col in cols:
        col = re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower()
        result.append(col)
    return result

def resolve_contract_columns(df):
    con_ids = []
    for col in df.columns:
        if ".con_id" in col:
            con_ids.append(col)
    df["resolved_con_id"] = df[con_ids].apply(lambda row: int(row.values[row.notna()][0]), axis=1)
    return df
    
def gen_positions_df(positions):
    df = pd.DataFrame(positions)
    cdf = pd.json_normalize(pd.DataFrame(util.tree(positions)).contract)
    df[cdf.columns] = cdf
    df.drop("contract", inplace=True, axis=1)
    df.columns = format_col_names(df.columns)
    return df

def add_on_pnl(df, pnl_singles):
    for i, row in df.iterrows():
        con_id = row["resolved_con_id"]
        pnl_single = list(filter(lambda x: x.conId == con_id, pnl_singles))[0]
        df.at[row.name, 'pnl_daily'] = pnl_single.dailyPnL
        df.at[row.name, 'pnl_net'] = pnl_single.unrealizedPnL
    return df

df = gen_positions_df(positions)
df = resolve_contract_columns(df)

df['pnl_daily'] = None
df['pnl_net'] = None

df = add_on_pnl(df, ib.pnlSingle())

df[['pnl_daily', 'pnl_net']].sum()

In [ ]:
ib.disconnect()